<a href="https://colab.research.google.com/github/ChapelFob80930/AI-Generated-Text-Detector/blob/main/AITextTest2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

train_prompts = pd.read_csv("https://raw.githubusercontent.com/ChapelFob80930/AI-Generated-Text-Detector/refs/heads/main/train_prompts.csv")
print(train_prompts.head())
print("\n")
train_essays = pd.read_csv("https://raw.githubusercontent.com/ChapelFob80930/AI-Generated-Text-Detector/refs/heads/main/train_essays.csv")
print(train_essays.head())
print("\n")
test_essays = pd.read_csv("https://raw.githubusercontent.com/ChapelFob80930/AI-Generated-Text-Detector/refs/heads/main/test_essays.csv")
print(test_essays.head())
# ai_human_dataset = pd.read_csv("/content/AI_Human.csv")
# print(ai_human_dataset.head())

   prompt_id                       prompt_name  \
0          0                   Car-free cities   
1          1  Does the electoral college work?   

                                        instructions  \
0  Write an explanatory essay to inform fellow ci...   
1  Write a letter to your state senator in which ...   

                                         source_text  
0  # In German Suburb, Life Goes On Without Cars ...  
1  # What Is the Electoral College? by the Office...  


         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated  
0          0  
1          0  
2    

In [2]:

import nltk
nltk.download()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_eng to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package averaged_perceptron_tagger_rus to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_rus.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package bcp47 to /root/nltk_d


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')

def lemmatize_text(text):
  doc = nlp(" ".join(text))
  return [token.lemma_ for token in doc]

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def _remove_stopwords_(texts):
  return [text for text in texts if text.isalnum() and text not in stop_words]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def preprocess_text(series):
    def process(text):
        tokens = word_tokenize(text)
        tokens = _remove_stopwords_(tokens)
        tokens = lemmatize_text(tokens)
        return " ".join(tokens)
    return series.apply(process)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
train_essays["preprocessedText"]=preprocess_text(train_essays['text'])

In [7]:
print(train_essays.head())

         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated                                   preprocessedText  
0          0  car Cars around since become famous 1900s Henr...  
1          0  transportation large necessity country worldwi...  
2          0  America love affair vehicle seem cool say Elis...  
3          0  how often ride car do drive one motor vehicle ...  
4          0  car wonderful thing they perhaps one world gre...  


In [8]:
from gensim.models import Word2Vec

sentences = train_essays['preprocessedText'].apply(lambda x: x.split())

word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

word_vector = word2vec_model.wv['car']
print("Vector for 'car':", word_vector)

# train_essays['word2vecVector'] = sentences.apply(
#     lambda x: sum([word2vec_model.wv[word] for word in x if word in word2vec_model.wv]) / len(x)
# )

def calculate_average_word_vector(words,model,vector_size=100):
    word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(vector_size)

train_essays['word2vecVector'] = train_essays['preprocessedText'].apply(lambda x: calculate_average_word_vector(x.split(),word2vec_model))


Vector for 'car': [ 0.9798277  -0.21653482 -0.85568213  1.1064807  -0.44638282 -2.220694
  3.2047741   0.26494384 -0.93324506  0.99873567  0.33852425  1.3901498
  0.1737627   1.6806521  -0.5208178  -1.8023901   0.33743605 -1.1215509
 -0.9847663  -2.6631947   2.8432589   0.16027     0.44361705 -0.40433502
 -0.54924816 -1.4057215  -0.6584818  -0.13003407 -1.0804116  -0.883335
 -0.5462925   0.8330227   0.17081156  0.81720877  0.7923248  -0.48373425
  0.38424653 -0.84581417  0.5594386  -2.2986102  -0.59632707  0.3130125
 -0.44109207  0.17779213 -0.25210264 -0.06692018 -0.10237622  0.84020907
  2.529846    1.4646838   0.6271458   1.5595932   0.3636652   0.90081567
  1.4002857  -1.190885    1.1704539  -0.8751411  -2.3014438   0.55963564
  0.27354223 -0.23019417  0.03099069 -2.2606435  -0.75954056  0.9766482
  0.19118032  0.45691997  0.01427387  0.17866276  0.5567877   0.09354579
 -0.27052915 -1.765663    1.5796745  -0.8201634   2.6097815   0.6628401
 -1.7131293  -0.3275655   0.1442206   0.96

In [9]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tagged_data = [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(train_essays['preprocessedText'])]

doc2vec_model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=1, workers=4)

train_essays['doc2vecVector'] = train_essays.index.map(lambda x: doc2vec_model.dv[str(x)])


In [10]:
print(train_essays.head())

         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated                                   preprocessedText  \
0          0  car Cars around since become famous 1900s Henr...   
1          0  transportation large necessity country worldwi...   
2          0  America love affair vehicle seem cool say Elis...   
3          0  how often ride car do drive one motor vehicle ...   
4          0  car wonderful thing they perhaps one world gre...   

                                      word2vecVector  \
0  [0.05613463, 0.01011937, -0.51041126, -0.31128...   
1  [0.106938384, 0.034

In [11]:
from sklearn.model_selection import train_test_split

y = train_essays['generated']

train_index, test_index = train_test_split(train_essays.index, test_size=0.2, random_state=42)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_tfidf = tfidf_vectorizer.fit_transform(train_essays['preprocessedText'])

X_train_tfidf, X_test_tfidf = X_tfidf[train_index], X_tfidf[test_index]
y_train, y_test = y[train_index], y[test_index]


In [13]:
train_essays['word2vecVector'] = train_essays['word2vecVector'].apply(lambda x: np.array(x))

X_word2vec = np.stack(train_essays['word2vecVector'].values)

X_train_word2vec, X_test_word2vec = X_word2vec[train_index], X_word2vec[test_index]

print(X_train_word2vec.shape)
print(X_test_word2vec.shape)
print(X_train_word2vec)

(1102, 100)
(276, 100)
[[ 0.116435   -0.00916288 -0.4983675  ... -0.49612486  0.27139837
  -0.06419957]
 [-0.01954295  0.19030935 -0.33091646 ... -0.25617954  0.13824639
   0.04567866]
 [-0.7795138   0.47796938  0.65176296 ... -0.47629836  0.05644946
  -0.0418004 ]
 ...
 [-0.01678405  0.02067794 -0.32247236 ... -0.5874986   0.24766058
  -0.2081457 ]
 [-0.89122987  0.4731477   0.7921602  ... -0.40872023 -0.06214165
   0.02992062]
 [-0.02398463  0.32435992 -0.33321595 ... -0.21148597  0.07829262
  -0.08271158]]


In [16]:
train_essays['doc2vecVector'] = train_essays['doc2vecVector'].apply(lambda x: np.array(x))
X_doc2vec = np.stack(train_essays['doc2vecVector'].values)
X_train_doc2vec, X_test_doc2vec = X_doc2vec[train_index], X_doc2vec[test_index]
print(X_train_doc2vec.shape)
print(X_test_doc2vec.shape)
print(X_train_doc2vec)

(1102, 100)
(276, 100)
[[-0.15162046 -0.3150435  -0.39042804 ... -0.13023089  0.4106068
   0.01839693]
 [-0.41978338 -0.16028115 -0.8079136  ... -0.06477406  0.25259474
   0.15097798]
 [-0.6349936   0.06294966  0.2266118  ... -0.34687087  0.1240015
   0.29605973]
 ...
 [-0.2682556  -0.23802033 -0.43457854 ... -0.67251056  0.6902878
  -0.13823205]
 [-0.49591488  0.18285985  0.09484906 ... -0.32769707  0.14330235
   0.00381349]
 [-0.28169638 -0.13951762 -0.39002886 ...  0.10226408  0.08423739
   0.03543597]]


In [17]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def evaluate_model(X_train, X_test, y_train, y_test):
    svm_model = SVC(class_weight='balanced')
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred,),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }


results_tfidf = evaluate_model(X_train_tfidf, X_test_tfidf, y_train, y_test)
print("TF-IDF Results:", results_tfidf)


results_word2vec = evaluate_model(X_train_word2vec, X_test_word2vec, y_train, y_test)
print("Word2Vec Results:", results_word2vec)


results_doc2vec = evaluate_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test)
print("Doc2Vec Results:", results_doc2vec)


TF-IDF Results: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}
Word2Vec Results: {'Accuracy': 0.9891304347826086, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}
Doc2Vec Results: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def evaluate_model(X_train, X_test, y_train, y_test):
    svm_model = SVC(class_weight='balanced')
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred,),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }


results_tfidf = evaluate_model(X_train_tfidf, X_test_tfidf, y_train, y_test)
print("TF-IDF Results:", results_tfidf)


results_word2vec = evaluate_model(X_train_word2vec, X_test_word2vec, y_train, y_test)
print("Word2Vec Results:", results_word2vec)


results_doc2vec = evaluate_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test)
print("Doc2Vec Results:", results_doc2vec)


TF-IDF Results: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}
Word2Vec Results: {'Accuracy': 0.9891304347826086, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}
Doc2Vec Results: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(X_train, X_test, y_train, y_test):
    svm_model = SVC(class_weight='balanced')
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred,),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }


results_tfidf = evaluate_model(X_train_tfidf, X_test_tfidf, y_train, y_test)
print("TF-IDF Results:", results_tfidf)


results_word2vec = evaluate_model(X_train_word2vec, X_test_word2vec, y_train, y_test)
print("Word2Vec Results:", results_word2vec)


results_doc2vec = evaluate_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test)
print("Doc2Vec Results:", results_doc2vec)


TF-IDF Results: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}
Word2Vec Results: {'Accuracy': 0.9891304347826086, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}
Doc2Vec Results: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def evaluate_model(X_train, X_test, y_train, y_test):
    svm_model = SVC(class_weight='balanced')
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }


results_tfidf = evaluate_model(X_train_tfidf, X_test_tfidf, y_train, y_test)
print("TF-IDF Results:", results_tfidf)


results_word2vec = evaluate_model(X_train_word2vec, X_test_word2vec, y_train, y_test)
print("Word2Vec Results:", results_word2vec)


results_doc2vec = evaluate_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test)
print("Doc2Vec Results:", results_doc2vec)


TF-IDF Results: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}
Word2Vec Results: {'Accuracy': 0.9891304347826086, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}
Doc2Vec Results: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
